In [8]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import wandb
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [9]:
print('CUDA GPU available : {}'.format(torch.cuda.is_available()))
try:
    print('{} GPU(s) is(are) allocated'.format(torch.cuda.device_count()))
except:
    print('GPUs are not allocated. Current runtime is on CPU.')
device = torch.device("cuda")

CUDA GPU available : True
1 GPU(s) is(are) allocated


In [10]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'
train_dir = '/opt/ml/input/data/train'

## Dataset

In [11]:
class TrainDataset(Dataset):
    def __init__(self, train_dir, is_Train=True, transform=None):
        super().__init__()
        
        csv_path = os.path.join(train_dir, 'train.csv')
        csv = pd.read_csv(csv_path)
        self.image_dir = os.path.join(train_dir, 'images')
        self.transform = transform
        self.image_path = []
        path = csv['path']
        
        for p in path:
            images = [os.path.join(*[self.image_dir, p, image]) for image in os.listdir(os.path.join(self.image_dir, p)) if not image[:1] == '.']
            for image in images:
                self.image_path.append(image)
                
        self.comb_dic = {}
        comb = [(m, g, a) for m in ['m', 'i', 'n'] for g in ['male', 'female'] for a in [0, 1, 2]]
        for i, (m, g, a) in enumerate(comb):
            self.comb_dic[(m, g, a)] = i

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, idx):
        image_name = self.image_path[idx]
        image = Image.open(image_name)
        
        features = image_name.split('/')[-2:]
        mask = features[1][0]
        age = int(features[0].split('_')[-1])
        gender = features[0].split('_')[1]
        
        if age >= 60:
            age = 2
        elif age >= 30:
            age = 1
        else:
            age = 0        

        target = self.comb_dic[(mask, gender, age)]
        
        if self.transform:
            image = self.transform(image)
        
        return image, target

class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [12]:
from sklearn.metrics import confusion_matrix

def check_f1(loader, model, device):
    model.eval()
    y_true = []
    y_predicted = []
    with torch.no_grad():
        for x, y in loader:
            y_true += y.tolist()

            x = x.to(device=device)
            y = y.to(device=device)


            scores = model(x)
            _, predictions = scores.max(1)
            y_predicted += predictions.tolist()

    cm = confusion_matrix(y_true, y_predicted)
    F1 = []
    for c in range(18):
        precision = cm[c][c] / np.sum(cm, axis=0)[c]
        recall = cm[c][c] / np.sum(cm, axis=1)[c]
        F1.append(2 * precision * recall / (precision + recall))
    macro_F1 = np.mean(F1)
    wandb.log({"F1" : macro_F1})
    print("Confusion Matrix")
    for row in cm:
        print(row)
    print("Validation F1 score :" , macro_F1)
    print("-"*50)
    model.train()

In [13]:
from torch.utils.data.dataset import random_split
tfms = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

dataset = TrainDataset(train_dir, transform=tfms)
train_dataset, val_dataset = random_split(dataset, [int(len(dataset)*0.8),int(len(dataset)*0.2)])
print(len(dataset))
# plt.imshow(np.array(train[1][0].permute(1,2,0)))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=1, drop_last=False)
val_loader   = DataLoader(dataset=val_dataset, batch_size=32)

54900


## Model

In [15]:
model = timm.create_model('nfnet_f5', num_classes=18).to(device)

In [19]:
import torch.optim as optim
lr = 3e-4

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.05)

In [20]:
wandb.init()


CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



In [21]:
NUM_EPOCHS = 10
VIS = 30

for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % VIS == VIS-1:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / VIS))
            wandb.log({"loss" : running_loss})
            
            running_loss = 0.0
            
    check_f1(val_loader, model, device)
            
    if epoch < NUM_EPOCHS:
        scheduler.step()
print("Finish")

[1,    30] loss: 1.377
[1,    60] loss: 1.213
[1,    90] loss: 1.087
[1,   120] loss: 0.956
[1,   150] loss: 0.995
[1,   180] loss: 0.911
[1,   210] loss: 0.954
[1,   240] loss: 0.829
[1,   270] loss: 0.800
[1,   300] loss: 0.838
[1,   330] loss: 0.829
[1,   360] loss: 0.784
[1,   390] loss: 0.756
[1,   420] loss: 0.712
[1,   450] loss: 0.730
[1,   480] loss: 0.715
[1,   510] loss: 0.805
[1,   540] loss: 0.663
[1,   570] loss: 0.734
[1,   600] loss: 0.614
[1,   630] loss: 0.677
[1,   660] loss: 0.723
[1,   690] loss: 0.586
[1,   720] loss: 0.614
[1,   750] loss: 0.578
[1,   780] loss: 0.647
[1,   810] loss: 0.522
[1,   840] loss: 0.568
[1,   870] loss: 0.469
[1,   900] loss: 0.542
[1,   930] loss: 0.505
[1,   960] loss: 0.558
[1,   990] loss: 0.506
[1,  1020] loss: 0.506
[1,  1050] loss: 0.494
[1,  1080] loss: 0.586


KeyboardInterrupt: 

## Inference

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = TestDataset(image_paths, tfms)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
# device = torch.device('cuda')

model.eval()
device = torch.device("cuda:0")
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

In [ ]:
model_path = '/opt/ml/weights/'
torch.save(model.state_dict(), model_path + 'model.pt')

In [ ]:
len(dataset)

In [ ]:
for 